In [ ]:
#import cx_Oracle
import pandas as pd
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError

<h1>Pobieranie danych z bazy</h1>

In [ ]:
data = {}

dbtables = ['MEB_DGM', 'MEB_DMC', 'MEB_GROB', 'MEB_KO', 'MEB_KO_DGM', 'MEB_KO_RODZAJ', 'MEB_KO_STREFA', 'MEB_KS', 'ONI_CIRCUITS']
querys = [
#Tutaj występują duplikaty, ale jeszcze nie zdecydowaliśmy co z nimi zrobić
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY DMC ORDER BY ID DESC) AS rn
  FROM
    Z3DMC.MEB_DGM t
) subquery
WHERE rn = 1
""",
#Tutaj występują duplikaty, ale jeszcze nie zdecydowaliśmy co z nimi zrobić, usunąłem tylko nulle i anomalię
#Po wykonaniu LEFT JOIN liczba rekordów się zwiększyła, trzeba usunąć duplikaty z DMC_CASTING ~ LO
# Zapytanie dodatkowo nie pobiera wartości null z DMC_CASTING
"""
SELECT *
FROM Z3DMC.MEB_DMC
WHERE (DMC <> 'null') AND (DMC <> 'NIECZYTELNY') AND (DMC_CASTING IS NOT NULL)
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_MEB_GROB DESC) AS rn
  FROM
    Z3DMC.MEB_GROB t
) subquery
WHERE rn = 1
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KO DESC) AS rn
  FROM
    Z3DMC.MEB_KO t
) subquery
WHERE rn = 1
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KO DESC) AS rn
  FROM
    Z3DMC.MEB_KO_DGM t
  WHERE ID_DMC <> 0
) subquery
WHERE rn = 1
""",
#Tabela słownik, niczego nie usuwamy, wczytuję całą tabelę
"""
SELECT *
FROM Z3DMC.MEB_KO_RODZAJ
""",
#Tabela słownik, niczego nie usuwamy, wczytuję całą tabelę
"""
SELECT *
FROM Z3DMC.MEB_KO_STREFA
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KS DESC) AS rn
  FROM
    Z3DMC.MEB_KS t
) subquery
WHERE rn = 1
""",
#Tabela ONI_CIRCUITS posiada wiele duplikatów, a powinno być ich do 28 per jeden ID_DMC, dlatego wrzucam całą tabelę i przegadamy co z tym zrobić.
#Poniżej zakomentowany kod usuwający wszystkie duplikaty.
"""
SELECT ID_DMC, CIRCUIT_NR, 
MAX(ASSIGMENT) AS ASSIGMENT, 
MAX(FLOW) AS FLOW, 
MAX(SET_POINT) AS SET_POINT,
MAX(START_DELAY) AS START_DELAY,
MAX(TEMP) AS TEMP,
MAX(WORKING_MODE) AS WORKING_MODE
FROM Z3DMC.ONI_CIRCUITS
GROUP BY ID_DMC, CIRCUIT_NR
ORDER BY ID_DMC
"""
#SELECT *
#FROM (
#  SELECT
#    t.*,
#    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_CIRCUIT DESC) AS rn
#  FROM
#    Z3DMC.ONI_CIRCUITS t
#) subquery
#WHERE rn = 1
]
username = 'dmcro'
password = 'ouZ9Rys7astDp4byjUvO'
dsn = 'vwpnwrud0000rac:1521/dmc.vwg'
dbhostname='vwpnwrud0000rac'
service_name='dmc.vwg'

In [ ]:
try:
    sqlalchemy_engine="oracle+cx_oracle://"+username+":"+password+"@"+dbhostname+"/?service_name="+service_name
    engine = sqlalchemy.create_engine(sqlalchemy_engine, arraysize=1000)
    for table, query in zip(dbtables, querys):
        data.update({table: pd.read_sql(query, engine)})
except SQLAlchemyError as e:
    print(e)

<h1>Usuwanie zbędnych kolumn z tabel</h1>

In [ ]:
dbtables = ['MEB_DGM', 'MEB_DMC', 'MEB_GROB', 'MEB_KO', 'MEB_KO_DGM', 'MEB_KS']
columns = [['timestamp', 'data_znakowania', 'data_odlania', 'metal_level', 'metal_pressure'],                                           #MEB_DGM
['timestamp', 'update_time','id_meb_containers', 'packed_time', 'first_packed_time', 'production_step', 'status_koncowy'],              #MEB_DMC
['id_meb_grob', 'shift_number', 'last_operation', 'timestamp', 'production_date', 'reworkrequested',                                    
'reworkdone', 'partcleaningisfinished', 'waitfortoolcheck', 'workingstep1', 'workingstep2', 'workingstep3', 'workingstep4', 'mms_ok'],  #MEB_GROB
['id_ko', 'data', 'timestamp', 'eks'],                                                                                                  #MEB_KO
['id_ko','data_odlania', 'timestamp', 'operator'],                                                                                      #MEB_KO_DGM
['id_ks', 'nrgniazda', 'liczbawystapien', 'nrformy', 'data', 'timestamp', 'gradedmc_max','gradedmc_aktualny']]                          #MEB_KS

for table, column in zip(dbtables, columns):
    data[table].drop(columns=column, inplace=True)
    #print(f'{table}: {data[table].columns}')

<h1>Łączenie tabel</h1>
Doklejać do MEB_DGM

In [ ]:
# usuwanie znaków białych z DMC[MEB_DGM] i DMC_CASTING[MEB_DMC]
data['MEB_DMC'].dmc_casting = data['MEB_DMC']['dmc_casting'].str.strip()
data['MEB_DGM'].dmc = data['MEB_DGM']['dmc'].str.strip()

In [ ]:
# usuwanie z meb_dmc wierszy z 'WORKPIECE NIO' w kodzie DMC
data['MEB_DMC'] = data['MEB_DMC'][~data['MEB_DMC']['dmc'].str.contains('WORKPIECE', case=False, na=False)]

In [ ]:
# wybieranie rekordów dla MEB+ 
data['MEB_DGM'] = data['MEB_DGM'][(data['MEB_DGM']['nr_dgm'].between(8, 10)) & (data['MEB_DGM']['dmc'].apply(lambda x: len(x)) == 21)]
# usunięcie anomalii z MEB_DMC
data['MEB_DMC'] = data['MEB_DMC'][data['MEB_DMC']['dmc'].str[:3] == '0MH']

In [ ]:
# łączę tabele MEB_KO i MEB_KO_DGM z tabelami MEB_KO_STREFA/RODZAJ

data['MEB_KO'] = data['MEB_KO'].merge(data['MEB_KO_STREFA'], left_on='nok_strefa', right_on='indeks', how='inner')
data['MEB_KO'].drop(columns=['indeks'], inplace=True)

data['MEB_KO'] = data['MEB_KO'].merge(data['MEB_KO_RODZAJ'], left_on='nok_rodzaj', right_on='indeks', how='inner')
data['MEB_KO'].drop(columns=['indeks'], inplace=True)

data['MEB_KO_DGM'] = data['MEB_KO_DGM'].merge(data['MEB_KO_STREFA'], left_on='nok_strefa', right_on='indeks', how='inner')
data['MEB_KO_DGM'].drop(columns=['indeks'], inplace=True)

data['MEB_KO_DGM'] = data['MEB_KO_DGM'].merge(data['MEB_KO_RODZAJ'], left_on='nok_rodzaj', right_on='indeks', how='inner')
data['MEB_KO_DGM'].drop(columns=['indeks'], inplace=True)

In [ ]:
# łączę tabelę MEB_DMC z tabelą MEB_KO
data['MEB_DMC'] = data['MEB_DMC'].merge(data['MEB_KO'], on='id_dmc', how='left')
data['MEB_DMC'].drop(columns=['rn'], inplace=True)

In [ ]:
# łączę tabelę MEB_DMC z tabelą MEB_GROB
data['MEB_DMC'] = data['MEB_DMC'].merge(data['MEB_GROB'], on='id_dmc', how='left')
data['MEB_DMC'].drop(columns=['rn'], inplace=True)

In [ ]:
# łączę tabelę MEB_DMC z tabelą MEB_KS
data['MEB_DMC'] = data['MEB_DMC'].merge(data['MEB_KS'], on='id_dmc', how='left')
data['MEB_DMC'].drop(columns=['rn'], inplace=True)

In [ ]:
# przygotowywuję tabelę ONI_CIRCUITS do połączenia 
oni_circuits = data['ONI_CIRCUITS'].pivot(index='id_dmc', columns='circuit_nr', values=['assigment', 'flow', 'set_point', 'start_delay', 'working_mode', 'temp'])
oni_circuits.columns = oni_circuits.columns.map('{0[0]}_{0[1]}'.format) 
oni_circuits.reset_index(inplace=True)

In [ ]:
final_table = data['MEB_DGM'].copy()
final_table.drop(columns=['rn'], inplace=True)
# łączę tabelę MEB_DGM z tabelą MEB_KO_DGM
final_table = final_table.merge(data['MEB_KO_DGM'], left_on='id', right_on='id_dmc', how='left')
final_table.drop(columns=['rn'], inplace=True)
# łączę z tabelą MEB_DGM
final_table = final_table.merge(oni_circuits, left_on='id', right_on='id_dmc', how='inner')
final_table.drop(columns=['id_dmc_y'], inplace=True)
final_table.rename(columns={'id_dmc_x': 'id_dmc'}, inplace=True)
# łączę tabelę MEB_DMC z ONI_CIRCUITS
final_table = final_table.merge(data['MEB_DMC'], left_on='dmc', right_on='dmc_casting', how='left', suffixes=('_DGM', '_DMC'))

In [ ]:
duplicate_count_oni = final_table['dmc_DMC'].duplicated(keep=False).sum()
print(f"Number of rows with the same 'dmc' value: {duplicate_count_oni}")
final_table.drop(columns=['nok_strefa_DGM', 'nok_rodzaj_DGM', 'status_ko_DGM', 'kod_pola_DGM', 'rodzaj_uszkodzenia_DGM'], inplace=True)
final_table.rename(columns={'nok_strefa_DMC': 'nok_strefa', 'nok_rodzaj_DMC': 'nok_rodzaj', 
                            'status_ko_DMC': 'status_ko', 'kod_pola_DMC': 'kod_pola', 
                            'rodzaj_uszkodzenia_DMC': 'rodzaj_uszkodzenia'}, inplace=True)
                            
final_table.drop(index=final_table[(final_table['dmc_DGM'].duplicated(keep=False)) & (~final_table['dmc_casting'].isna())].index, inplace=True)
final_table.drop(columns = ['part_status'], inplace = True)

<h1>Usuwanie niepotrzebnych danych z tabeli koncowej</h1>

In [ ]:
final_table = final_table[~final_table['status'].isin(['4', '5', '7', '8', '10', '11'])]
final_table['status'] = final_table['status'].replace(['3', '14'], '2')
final_table = final_table.loc[~final_table['status_ko'].isin([0, 106])]
final_table = final_table.loc[~final_table['statusszczelnosc'].isin([0, 3])]
final_table = final_table.loc[~final_table['statusdmc'].isin([0])]

In [ ]:
print(list(final_table.keys()))

In [ ]:
for i in range(1,29):
    print(final_table[f'ref_time_{i}'].value_counts())
    print('\n')

In [ ]:
for i in range(1,29):
    print(final_table[f'start_delay_{i}'].value_counts())
    print('\n')

In [ ]:
for i in range(1,29):
    print(final_table[f'working_mode_{i}'].value_counts())
    print('\n')

<h1>Tworzenie statusu końcowego</h1>

In [ ]:
# status:
#   1! - IO, 3! - NIO DGM, 4 - NIO DMC, 5 - NIO stolik pelen, 7 - NIO prasa, 8 - NIO znakowarka, 10 - NIO AT, 11 - NIO znakowarka zajęta, 14! - NIO vacuum
#   106 - Pomiary niszczące 110 - do pomiaru ręcznego, 
# rodzaj_kontroli
# status_koncowy - WYWALONY
# status_ko
# part_status - zostalo wywalone clkowicie bo mialo same 1 
# statusszczelnosc - 3 - do impregnacji - do wywalenia
# statusdmc
print(final_table['status'].value_counts().sort_index())
print(final_table['status_ko'].value_counts())
print(final_table['statusszczelnosc'].value_counts())
print(final_table['statusdmc'].value_counts())

In [ ]:
final_table['our_final_status'] = final_table.apply(lambda row: max(int(row['status']), row['status_ko'], row['statusszczelnosc'], row['statusdmc']), axis=1)
print(final_table['our_final_status'].value_counts())

In [ ]:
final_table.drop(columns=['status', 'status_ko', 'statusszczelnosc', 'statusdmc'], inplace=True)

<h1>Eliminacja NaN</h1>

In [ ]:
# podmiana zer na NaNy, na następnym spotkaniu ustalimy co z tym zrobić
final_table.iloc[:, 28:196] = final_table[final_table.iloc[:, 28:196] != 0]
final_table.iloc[:, 28:196]

<h1>Zebranie kolumn do dalszej dyskucji</h1>

In [ ]:
# wyświetlenie kolumn z danymi numerycznymi
pd.set_option('display.max_columns', None)
final_table.select_dtypes(include=['number'])
# kolumny, które będą niepotrzebne: id, nr_dgm, status_ko, part_type, nrprogramu, id_dmc_DGM, id_dmc_DGM
# kolumny, o które trzeba zapytać produkcję: nachdruck_hub, anguss, max_press_kolbenhub, last_machine_number, last_pcf_number, machine_nr,
# cisnienie_tloka, rodzaj_kontroli, assigment

In [ ]:
# wyświetlanie danych kategorycznych
final_table.select_dtypes(include=['object'])
# kolumny do usunięcia: dmc_DGM, product_id, line_id, dmc_DMC, dmc_casting
# z jakiegoś powodu assigment 2,4,28 i flow 1 są danymi typu object - prawdopodobnie dlatego, że wszystkie wartości w kolumnach były równe 0 i zostały zmienione na NaN

In [ ]:
# DO DYSKUCJI!
final_table.drop(columns=['id', 'nr_dgm', 'part_type', 'nrprogramu', 'id_dmc_DGM', 'id_dmc_DGM', 
                          'dmc_DGM', 'product_id', 'line_id', 'dmc_DMC', 'dmc_casting'
                          'flow_1', 'flow_3', 'flow_5', ], inplace=True)

<h1>Normalizacja danych</h1>

In [ ]:
# Przekształcenie danych kategorycznych
final_table['kod_pola'].value_counts()

In [ ]:
final_table = pd.get_dummies(final_table, columns=['kod_pola', 'rodzaj_uszkodzenia'], dummy_na=True, dtype='uint8')

In [ ]:
# Normalizacja danych numerycznych
